In [ ]:
!nvcc --help | grep sm_

        target. The options -dlto -arch=sm_NN will add a lto_NN target; if you want
        to only add a lto_NN target and not the compute_NN that -arch=sm_NN usually
        for '--gpu-architecture' may be a 'real' architecture (such as a sm_50),
        --gpu-architecture=sm_50' is equivalent to 'nvcc --gpu-architecture=compute_50
        --gpu-code=sm_50,compute_50'.
        -arch=all         build for all supported architectures (sm_*), and add PTX
        -arch=all-major   build for just supported major versions (sm_*0), plus the
        -arch=native      build for all architectures (sm_*) on the current system
        'native','sm_50','sm_52','sm_53','sm_60','sm_61','sm_62','sm_70','sm_72',
        'sm_75','sm_80','sm_86','sm_87','sm_89','sm_90','sm_90a'.
        (such as sm_50), and PTX code for the 'virtual' architecture (such as compute_50).
        For instance, '--gpu-architecture=compute_60' is not compatible with '--gpu-code=sm_52',
        features that are not present o

In [ ]:
!nvidia-smi

Tue Jul  8 21:32:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%%writefile sample_sort.cu
#include <iostream>
#include <chrono>
#include <unordered_set>
#include <vector>
#include <cuda.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <thrust/device_vector.h>
#include <thrust/sort.h>
#include <thrust/reduce.h>
#include <thrust/scan.h>
#include <thrust/iterator/constant_iterator.h>

using namespace std;
const int THRESHOLD=16;

void printArray(vector<int> &arr){
  for(int i=0;i<arr.size();i++) cout<<arr[i]<<" ";
  cout<<"\n";
}

int getMin(int a, int b){
  if(a<b) return a;
  return b;
}

vector<int> randSample(vector<int> &arr, int s){
  vector<int> ret;
  unordered_set<int> miSet;
  int arr_size=arr.size();
  while(miSet.size()<s){
    int r=rand()%arr_size;
    miSet.insert(r);
  }
  for(int indice:miSet) ret.push_back(arr[indice]);
  return ret;
}

vector<int> getPivotes(vector<int> &sample, int numPivotes){
  int sample_size=sample.size();
  vector<int> ret;
  if(numPivotes==0||sample_size==0) return ret;
  if(sample_size<=numPivotes) return ret;
  for(int i=1;i<=numPivotes;i++){
    int indice=i*sample_size/(numPivotes+1);
    if(indice>=sample_size) indice=sample_size-1;
    ret.push_back(sample[indice]);
  }
  return ret;
}

__global__ void assignToBuckets(int* arr, int* pivotes, int* bucket_ids, int size, int numPivotes){
  int indice=blockIdx.x*blockDim.x+threadIdx.x;
  if(indice>=size) return;
  int val=arr[indice];
  int inf=0,sup=numPivotes;
  while(inf<sup){
    int mid=(inf+sup)/2;
    if(val<=pivotes[mid]) sup=mid;
    else inf=mid+1;
  }
  bucket_ids[indice]=inf;
}

void sampleSort(vector<int> &arr, int p){
  int arr_size=arr.size();
  if(arr_size<=THRESHOLD){
    sort(arr.begin(),arr.end());
    return;
  }
  int s=getMin(arr_size,4*p*log2(arr_size));
  vector<int> sample=randSample(arr,s);
  if(sample.size()<=p-1){
    sort(arr.begin(),arr.end());
    return;
  }
  sort(sample.begin(),sample.end());
  vector<int> pivotes=getPivotes(sample,p-1);
  if(pivotes.empty()){
    sort(arr.begin(),arr.end());
    return;
  }
  thrust::device_vector<int> d_arr(arr.begin(),arr.end());
  thrust::device_vector<int> d_pivotes(pivotes.begin(),pivotes.end());
  thrust::device_vector<int> d_bucket_ids(arr_size);
  int hebras=256;
  int bloques=(arr_size+hebras-1)/hebras;
  assignToBuckets<<<bloques,hebras>>>(
    thrust::raw_pointer_cast(d_arr.data()),
    thrust::raw_pointer_cast(d_pivotes.data()),
    thrust::raw_pointer_cast(d_bucket_ids.data()),
    arr_size,
    p-1
  );
  cudaDeviceSynchronize();
  thrust::stable_sort_by_key(d_bucket_ids.begin(),d_bucket_ids.end(),d_arr.begin());
  thrust::device_vector<int> keys_out(p);
  thrust::device_vector<int> counts(p);
  auto rbk_end=thrust::reduce_by_key(
    d_bucket_ids.begin(),
    d_bucket_ids.end(),
    thrust::constant_iterator<int>(1),
    keys_out.begin(),
    counts.begin()
  );
  int num_buckets=rbk_end.first-keys_out.begin();
  thrust::device_vector<int> d_offsets(p+1);
  d_offsets[0]=0;
  thrust::inclusive_scan(
    counts.begin(),
    counts.begin()+num_buckets,
    d_offsets.begin()+1
  );
  for(int i=0;i<num_buckets;i++){
    int inicio=d_offsets[i];
    int fin=d_offsets[i+1];
    if(fin>inicio) thrust::sort(d_arr.begin()+inicio,d_arr.begin()+fin);
  }
  thrust::copy(d_arr.begin(),d_arr.end(),arr.begin());
}

int main(int argc, char** argv){
  srand(time(NULL));
  int size=atoi(argv[1]),p=atoi(argv[2]);
  vector<int> arr;
  for(int i=0;i<size;i++) arr.push_back(1+rand()%200);
  cout<<"Arreglo inicial: ";
  printArray(arr);
  sampleSort(arr,p);
  auto finish=chrono::high_resolution_clock::now();
  auto duration=chrono::duration_cast<chrono::nanoseconds>(finish - start).count();
  cout<<"Tiempo demorado en ordenar arreglo: "<<duration/1000000000.0<<" s\n";
  return 0;
}

Overwriting sample_sort.cu


In [ ]:
!nvcc -arch=sm_75 -O2 sample_sort.cu -o sample_sort

In [ ]:
!./sample_sort 64 5

Arreglo inicial: 121 125 122 65 9 25 176 145 17 43 22 148 13 198 112 61 37 116 136 5 198 111 135 18 105 127 98 177 160 66 93 32 190 166 96 150 190 24 46 158 18 68 58 30 65 169 90 54 36 177 58 33 39 144 3 96 22 100 72 182 165 116 165 107 
Arreglo ordenado: 3 5 9 13 17 18 18 22 22 24 25 30 32 33 36 37 39 43 46 54 58 58 61 65 65 66 68 72 90 93 96 96 98 100 105 107 111 112 116 116 121 122 125 127 135 136 144 145 148 150 158 160 165 165 166 169 176 177 177 182 190 190 198 198 
